In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import pathlib

p = pathlib.Path().resolve()

while True:
    if (p / ".git").exists():
        os.chdir(p)
        print("프로젝트 루트:", p)
        break

    if p.parent == p:
        raise RuntimeError("프로젝트 루트를 찾지 못함.")

    p = p.parent

프로젝트 루트: E:\AI_Project\codeit_ai_rfp_rag_assistant


In [3]:
from src.document_loader import get_hwp_text
test_file = "./Data/files/국민연금공단_2024년 이러닝시스템 운영 용역.hwp"

test = get_hwp_text(test_file)
print(test[:500])

2024년 이러닝시스템 운영 용역 제안요청서

2024년 이러닝시스템 운영 용역 제안요청서
 

청렴한 업무처리 및 부조리 신고
국민연금공단은 모든 계약업무를 투명하고 공정하게 처리하고 있습니다.
계약과정 및 계약이행 시 금품·향응·편의 제공을 요구하는 직원이 있거나, 지연·학연 등의 사유로 특혜를 제공하는 것을 보거나 들은 경우에는 공단 홈페이지 부조리신고센터로 신고하여 주시기 바랍니다.


부조리
신고센터
국민연금 홈페이지(www.nps.or.kr)  클린센터

레드휘슬(www.redwhistle.org)익명성 보장
2024. 1.

인사혁신실 인재개발부


목   차


 . 사업 안내	   	1
   1. 사업개요	   	1
   2. 사업목적	   	1
   3. 사업범위	   	1
   4. 기대효과	   	2
 . 제안요청 내용	   	3
   1. 사업 운영방향	   	3
  


In [ ]:
import os
from src.chunking import load_chunks, save_chunks, load_and_chunk_all_docs
from src.vectorstore import build_vector_store

CHUNK_PATH = "./Data/chunks.pkl"
FILES_PATH = "./Data/files"

# ---------------------------------------------------------
# 1) 청크 존재 여부 확인
# ---------------------------------------------------------
if os.path.exists(CHUNK_PATH):
    print("기존 chunks.pkl 발견 — 로드합니다.")
    chunks = load_chunks(CHUNK_PATH)
else:
    print("기존 chunks.pkl 없음 — 새로 생성합니다.")
    chunks = load_and_chunk_all_docs(FILES_PATH)
    save_chunks(chunks, CHUNK_PATH)
    print("새 chunks.pkl 저장 완료")

# ---------------------------------------------------------
# 2) 벡터스토어 생성 또는 로드
# ---------------------------------------------------------
vector_store = build_vector_store(chunks)

# ---------------------------------------------------------
# 3) retriever 생성
# ---------------------------------------------------------
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

print("청크 & 벡터스토어 준비완료")

📌 기존 chunks.pkl 발견 — 로드합니다.
📌 기존 벡터스토어 감지됨 — 새로 추가하지 않고 로드만 진행
🚀 청크 & 벡터스토어 준비완료!


In [5]:
for i, doc in enumerate(chunks[:3]):
    print(f"\n[{i+1}] 문서명: {doc.metadata.get('source', '미상')}")
    print(doc.metadata)


[1] 문서명: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp
{'source': '(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp', 'org': '(사)벤처기업협회', 'category': '2024년 벤처확인종합관리시스템 기능 고도화 용역사업 입찰공고', 'doc_type': '제안요청서', 'budget': '352000000.0', 'open_date': '2024-03-19 10:49:46', 'start_date': 'nan', 'end_date': '2024-04-09 15:00:00'}

[2] 문서명: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp
{'source': '(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp', 'org': '(사)벤처기업협회', 'category': '2024년 벤처확인종합관리시스템 기능 고도화 용역사업 입찰공고', 'doc_type': '제안요청서', 'budget': '352000000.0', 'open_date': '2024-03-19 10:49:46', 'start_date': 'nan', 'end_date': '2024-04-09 15:00:00'}

[3] 문서명: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp
{'source': '(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용

In [6]:
docs = vector_store.get(include=["metadatas", "documents"])
print(len(docs["documents"]))

6629


In [7]:
import unicodedata

target_keyword = "고려대"  # 찾고 싶은 키워드

docs = vector_store.get(include=["metadatas", "documents"], limit=99999)
found = 0

for i, (meta, content) in enumerate(zip(docs["metadatas"], docs["documents"])):
    src = unicodedata.normalize("NFC", str(meta.get("source", ""))).strip()

    # '이러닝' 또는 자모분리형 '이러닝' 모두 검색
    if target_keyword in src or "고려대" in src:
        print(f"🧾 {i}번 문서명: {src}")
        print(f"본문 길이: {len(content)}자")
        print(f"앞 500자 미리보기:\n{content[:500]}")
        print("="*80)
        found += 1

🧾 753번 문서명: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
본문 길이: 113자
앞 500자 미리보기:
제 안 요 청 서
고려대학교
차세대 포털·학사 정보시스템 구축 사업
2024. 7. 01
※ 본 자료는 제안내용의 설명을 위한 배포자료로, 이외 목적으로 무단복제, 전달 및 사용하는 행위를 금함.
-1-
🧾 754번 문서명: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
본문 길이: 1447자
앞 500자 미리보기:
목 차
Ⅰ. 사업개요 ·················································································································· 4
1. 사업 개요 ·············································································································· 4
2. 사업 배경 ·············································································································· 4
3. 사업 범위 ·············································································································· 5
4. 기대
🧾 755번 문서명: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
본문 길이: 1391자
앞 500자 미리보기:
2. 추진 일정 ············································································································ 35
3. 추진 체계 ·················································

In [9]:
from src.rag_graph import run_rag_graph

question = input()

result = run_rag_graph(question, vector_store, retriever)
print(result)

입력 질문: 한국어촌어항공단_한국어촌어항공단 경영관리시스템(ERP·GW) 기능 고도의 문서에서 요구사항 고유번호  COR-006의 요구사항 명칭이 뭔지?

[RETRIEVE] 재시도 번호: 0
 → Strategy: strict search (top_n=3)
📄 자동선택된 문서: 한국어촌어항공단_한국어촌어항공단 경영관리시스템(ERP·GW) 기능 고도.hwp (유사도 0.645)
[JUDGE RAW OUTPUT]
 {
  "score": 1.0,
  "reason": "답변은 제공된 발췌문과 메타데이터에 근거하여 작성되었으며, 문서 목차에 '요구사항 목록(11쪽)'·'요구사항 상세(13쪽)'가 존재함을 정확히 지적하고 발췌본에는 COR-006의 명칭이 포함되어 있지 않음을 명확히 밝혔습니다. 문서에 기재된 사업예산(397,000,000원) 등 메타데이터를 정확히 반영했고, 문서에 없는 상세 요구사항 명칭이나 추가 날짜·수치 등을 추측하거나 첨가하지 않았습니다. 따라서 평가 기준(문서 근거 기반, 추측 여부 부재, 메타데이터 반영)을 모두 충족합니다."
}
[DECISION] score=1.0, retry=1
 → GOOD (종료)
핵심 요약
- 제공된 문서 본문에서는 요구사항 고유번호 COR-006의 요구사항 명칭이 나타나 있지 않습니다.  
- 문서 목차에는 요구사항 목록(11쪽)과 요구사항 상세(13쪽)가 있어 해당 항목은 원문 내에 존재할 가능성이 있으나, 본문에 해당 내용은 포함되어 있지 않습니다.

세부 설명
- 제공된 텍스트는 사업개요와 목차(요구사항 목록·상세의 위치 포함)까지를 보여주고 있으나, 실제 요구사항 항목(COR-006 등)의 명칭이나 상세 내용은 발췌본에 포함되어 있지 않습니다.  
- 문서 목차에 따르면 요구사항 목록은 11쪽, 요구사항 상세는 13쪽에 수록되어 있으므로 COR-006의 명칭은 원문 전체의 해당 페이지에서 확인할 수 있습니다.  
- 따라서 현재 제공된 자료만으로는 COR-006의 요구사항 명칭을 특정할 수 없습니다(언